In [3]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd

/home/gautam/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Get all index names
index_names = es.indices.get_alias().keys()

# Print index names
for index_name in index_names:
    print(index_name)


videoss
videos
articles
semanticsearch
videos_streamming
videossnew
job_postings


/tmp/ipykernel_10772/1346631996.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  index_names = es.indices.get_alias().keys()


In [32]:
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Index name
index_name = "videossnew"

# Initialize variables for pagination
scroll = "1m"  # Keep the search context open for 1 minute
page_size = 1000  # Number of documents to fetch in each scroll request
search_query = {"query": {"match_all": {}}}

# Perform initial search to get the first page of results
search_result = es.search(index=index_name, body=search_query, scroll=scroll, size=page_size)
scroll_id = search_result["_scroll_id"]
hits = search_result["hits"]["hits"]

# Print each document from the first page
for hit in hits:
    print(hit["_source"])

# Fetch subsequent pages of results using scroll
while True:
    # Perform scroll request to get the next page of results
    scroll_result = es.scroll(scroll_id=scroll_id, scroll=scroll)
    hits = scroll_result["hits"]["hits"]

    # Break the loop if there are no more hits
    if not hits:
        break

    # Print each document from the current page
    for hit in hits:
        print(hit["_source"])


/tmp/ipykernel_10772/3029990718.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_result = es.search(index=index_name, body=search_query, scroll=scroll, size=page_size)


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [videossnew]', videossnew, index_or_alias)

In [41]:
import requests

class VideoSearchEngine:
    def __init__(self, index_name="videossnew", host="localhost", port=9200):
        self.es = Elasticsearch([{'host': 'elsticsearch-ma', 'port': 9200, 'scheme': 'http'}])
        self.index_name = index_name
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")

    def setup_index(self):
        if not self.es.indices.exists(index=self.index_name):
            mappings = {
                "properties": {
                    "title": {"type": "text"},
                    "embedding": {"type": "dense_vector", "dims": 384},
                    "url": {"type": "keyword"}
                }
            }

            self.es.indices.create(index=self.index_name, body={"mappings": mappings})
            print(f"Index '{self.index_name}' created.")
        else:
            print(f"Index '{self.index_name}' already exists.")


    def index_article(self):
        title = "kohli fastest century"
        embedding = self.embedder.encode(title)

        # Index the video with its embedding
        doc = {"title": title, "embedding": embedding.tolist(), "url": "https://wwe61.comm"}
        self.es.index(index=self.index_name, body=doc)



    def semantic_search(self, query, top_k=10):
        query_embedding = self.embedder.encode(query)
        search_body = {
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {"query_vector": query_embedding.tolist()}
                    }
                }
            }
        }
        response = self.es.search(index=self.index_name, body=search_body, size=top_k)
        hits = response["hits"]["hits"]
        return [{"title": hit["_source"]["title"], "url": hit["_source"]["url"]} for hit in hits]


In [42]:
# Example usage
search_engine = VideoSearchEngine()
search_engine.setup_index()



Index 'videossnew' created.


/tmp/ipykernel_10772/2100391735.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not self.es.indices.exists(index=self.index_name):
/tmp/ipykernel_10772/2100391735.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  self.es.indices.create(index=self.index_name, body={"mappings": mappings})
/tmp/ipykernel_10772/2100391735.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  self.es.indices.create(index=self.index_name, body={"mappings": mappings})


In [15]:
# Fetch videos from API
# videos = search_engine.fetch_data_from_api()
# print(videos)

In [43]:
# Index videos
search_engine = VideoSearchEngine()
search_engine.index_article()



/tmp/ipykernel_10772/2100391735.py:31: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index=self.index_name, body=doc)
/tmp/ipykernel_10772/2100391735.py:31: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  self.es.index(index=self.index_name, body=doc)


In [44]:
# Perform semantic search
query = "kohli fastest"
results = search_engine.semantic_search(query)

/tmp/ipykernel_10772/2100391735.py:48: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = self.es.search(index=self.index_name, body=search_body, size=top_k)
/tmp/ipykernel_10772/2100391735.py:48: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = self.es.search(index=self.index_name, body=search_body, size=top_k)


In [45]:
# Display results
print(f"Top {len(results)} videos related to '{query}':")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result['title']}: {result['url']}")

Top 1 videos related to 'kohli fastest':
1. kohli fastest century: https://wwe61.comm


In [9]:



# Perform semantic search
query =""
results = search_engine.semantic_search(query)



/tmp/ipykernel_37300/267053371.py:116: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = self.es.search(index=self.index_name, body=search_body, size=top_k)
/tmp/ipykernel_37300/267053371.py:116: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = self.es.search(index=self.index_name, body=search_body, size=top_k)


In [10]:
# Display results
print(f"Top {len(results)} articles related to '{query}':")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result['title']}: {result['url']}")


Top 10 articles related to '':
1. The Thrill of Extreme Sports: https://example.com/14
2. The Thrill of Extreme Sports: https://example.com/14
3. The Thrill of Extreme Sports: https://example.com/14
4. The Thrill of Extreme Sports: https://example.com/14
5. The Thrill of Extreme Sports: https://example.com/14
6. The Thrill of Extreme Sports: https://example.com/14
7. The Thrill of Extreme Sports: https://example.com/14
8. The Thrill of Extreme Sports: https://example.com/14
9. world cup final: https://example.com/6001
10. world cup final: https://example.com/6001
